# Aula 4 - Manipulação de df: groupby e merge


### Objetivos

Apresentar como unir dataframes e realizar cálculos com dados agrupados

____________________________

### Habilidades a serem desenvolvidas nessa aula

Ao final da aula o aluno deve:

- Saber como concatenar dataframes,
- Conseguir agrupar os dados e aplicar vários métodos à eles


____
____
____

## Titanic

O arquivo que usaremos hoje é relativo ao Titanic! Essa é uma das bases mais famosas de ciência de dados. Você pode saber mais sobre estes dados [clicando aqui!](https://www.kaggle.com/c/titanic)

In [9]:
import pandas as pd

In [267]:
# lê dataframe do arquivo titanic.csv 
df = pd.read_csv("data/titanic.csv")

E se quisessemos calcular a média de Fare por Pclasse utilizando apenas o que aprendemos até agora?

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df[df['Pclass']==1].Fare#.mean()

1      71.2833
3      53.1000
6      51.8625
11     26.5500
23     35.5000
        ...   
871    52.5542
872     5.0000
879    83.1583
887    30.0000
889    30.0000
Name: Fare, Length: 216, dtype: float64

In [5]:
df[df['Pclass']==2].Fare.mean()

20.662183152173913

In [6]:
df[df['Pclass']==3].Fare.mean()

13.675550101832993

Ou de forma mais automática:

In [7]:
df.Pclass.sort_values().unique()

array([1, 2, 3], dtype=int64)

In [8]:
for i in df.Pclass.sort_values().unique():
    print(f'Ticket médio da classe {i}: ', df[df['Pclass']==i].Fare.mean())

Ticket médio da classe 1:  84.1546875
Ticket médio da classe 2:  20.662183152173913
Ticket médio da classe 3:  13.675550101832993


E se quisessemos calcular a média por Pclass e Sex?

### Groupby
Assim como no SQL, no pandas também temos um método com o qual podemos agregar os dados. O `groupby` primeiro separa nossos dados em grupos definidos dentro do método,  aplica um tipo de operação usando agregação, transformação, filtragem ou até uma função própria e, por fim, junta os resultados encontrados.
<br>

<img src="groupby.png"  style="width: 700px" >

Exemplo de aplicação da função de agregação `mean`
<br><br><br>

Utilizar o `groupby` é o mesmo que fazer a sequência:

   1. Dividir os dados em grupos utilizando um critério
    
   2. Aplicar uma função em cada um dos grupos separadamente
    
   3. Combinar o resultado em uma estrutura de dados

#### Funções de agregação
Com essas funções podemos aplicar operações estatísticas nos nossos dados. Exemplos:<br>
`mean`, `std`, `max`, `min`, `count`, `sum`, `var`. <br>
Quando queremos aplicar apenas uma dessas operações podemos chamá-las diretamente após o `groupby`:


In [16]:
df.groupby(['Pclass']).Fare.mean()

Pclass
1    84.154687
2    20.662183
3    13.675550
Name: Fare, dtype: float64

In [12]:
df.groupby(['Pclass'])['Fare'].mean()

Pclass
1    84.154687
2    20.662183
3    13.675550
Name: Fare, dtype: float64

In [27]:
df.groupby(['Pclass'])[['Fare']].mean()

,Fare
Pclass,
1,84.154687
2,20.662183
3,13.675550


In [32]:
pd.options.display.float_format= "{:,.2f}".format

In [33]:
# Agrupa por Pclass e Sex e calcula a média de cada grupo
df.groupby(['Pclass', 'Sex']).mean()


PassengerId  Survived   Age  SibSp  Parch   Fare
Pclass Sex                                                     
1      female       469.21      0.97 34.61   0.55   0.46 106.13
       male         455.73      0.37 41.28   0.31   0.28  67.23
2      female       443.11      0.92 28.72   0.49   0.61  21.97
       male         447.96      0.16 30.74   0.34   0.22  19.74
3      female       399.73      0.50 21.75   0.90   0.80  16.12
       male         455.52      0.14 26.51   0.50   0.22  12.66

Aqui agregamos os dados por Pclass e Sex e em todas as colunas numéricas foi calculada a média. Se quiséssemos a média de apenas uma coluna poderíamos adicioná-la ao final da nossa sentença:

In [36]:
# Queremos apenas a média de idade considerando a classe e o sexo
df.groupby(["Pclass", "Sex"])[['Age']].mean()

Age
Pclass Sex         
1      female 34.61
       male   41.28
2      female 28.72
       male   30.74
3      female 21.75
       male   26.51

In [37]:
# Queremos apenas a média de idade considerando a classe e o sexo
df.groupby(["Pclass", "Sex"]).mean()[['Age']]

Age
Pclass Sex         
1      female 34.61
       male   41.28
2      female 28.72
       male   30.74
3      female 21.75
       male   26.51

Ou de modo mais eficiente:

In [38]:
df.groupby(["Pclass", "Sex"])[['Age', 'Fare']].mean()

Age   Fare
Pclass Sex                
1      female 34.61 106.13
       male   41.28  67.23
2      female 28.72  21.97
       male   30.74  19.74
3      female 21.75  16.12
       male   26.51  12.66

In [39]:
df.groupby(["Pclass", "Sex"])[['Age', 'Fare']].mean()

Age   Fare
Pclass Sex                
1      female 34.61 106.13
       male   41.28  67.23
2      female 28.72  21.97
       male   30.74  19.74
3      female 21.75  16.12
       male   26.51  12.66

In [40]:
df.groupby(["Pclass", "Sex"])[['Age', 'Fare']].mean().index

MultiIndex([(1, 'female'),
            (1,   'male'),
            (2, 'female'),
            (2,   'male'),
            (3, 'female'),
            (3,   'male')],
           names=['Pclass', 'Sex'])

Note que `df.groupby('A').colname.mean()` é mais eficiente que `df.groupby('A').mean().colname` pois a agregação só será realizada na coluna de interesse (colname).

Quando queremos aplicar mais de uma operação chamamos o método `.agg()`

In [41]:
df.groupby(["Pclass"]).agg(['mean','max','min']).columns

MultiIndex([('PassengerId', 'mean'),
            ('PassengerId',  'max'),
            ('PassengerId',  'min'),
            (   'Survived', 'mean'),
            (   'Survived',  'max'),
            (   'Survived',  'min'),
            (        'Age', 'mean'),
            (        'Age',  'max'),
            (        'Age',  'min'),
            (      'SibSp', 'mean'),
            (      'SibSp',  'max'),
            (      'SibSp',  'min'),
            (      'Parch', 'mean'),
            (      'Parch',  'max'),
            (      'Parch',  'min'),
            (       'Fare', 'mean'),
            (       'Fare',  'max'),
            (       'Fare',  'min')],
           )

In [42]:
df.groupby(["Pclass"]).agg(['mean','max','min'])

PassengerId          Survived           Age            SibSp          \
              mean  max min     mean max min  mean   max  min  mean max min   
Pclass                                                                        
1           461.60  890   2     0.63   1   0 38.23 80.00 0.92  0.42   3   0   
2           445.96  887  10     0.47   1   0 29.88 70.00 0.67  0.40   3   0   
3           439.15  891   1     0.24   1   0 25.14 74.00 0.42  0.62   8   0   

       Parch          Fare              
        mean max min  mean    max  min  
Pclass                                  
1       0.36   4   0 84.15 512.33 0.00  
2       0.38   3   0 20.66  73.50 0.00  
3       0.39   6   0 13.68  69.55 0.00

Para operações distintas em colunas distintas passamos um dicionário com o nome da coluna como chave e a operação como valor

In [47]:
import numpy as np
df.groupby(['Pclass']).agg({'Embarked': pd.Series.mode, 'Fare': 'mean'})

,Embarked,Fare
Pclass,,
1,S,84.15
2,S,20.66
3,S,13.68


Reparem que a coluna utilizada no `groupby` virou um index do nosso df. Para convertê-la em coluna novamente temos duas formas: <br>
  1. chamar o parâmetro `as_index=False` dentro do `groupby`
  2. aplicar `.reset_index()` ao final da sentença

In [48]:
df.groupby(['Pclass']).agg({'Embarked': pd.Series.mode, 'Fare': 'mean'}).index

Int64Index([1, 2, 3], dtype='int64', name='Pclass')

In [49]:
type(df.groupby(['Pclass']).agg({'Embarked': pd.Series.mode, 'Fare': 'mean'}))

pandas.core.frame.DataFrame

In [50]:
# exemplo com as_index = False
df.groupby(['Pclass'], as_index=False).agg({'Embarked': pd.Series.mode, 'Fare': 'mean'})

,Pclass,Embarked,Fare
0,1,S,84.15
1,2,S,20.66
2,3,S,13.68


In [51]:
# exemplo com .reset_index()
df.groupby(['Pclass']).agg({'Embarked': pd.Series.mode, 'Fare': 'mean'}).reset_index()

,Pclass,Embarked,Fare
0,1,S,84.15
1,2,S,20.66
2,3,S,13.68


In [52]:
df.groupby(["Pclass", "Sex"])[['Age', 'Fare']].mean().reset_index()

,Pclass,Sex,Age,Fare
0,1,female,34.61,106.13
1,1,male,41.28,67.23
2,2,female,28.72,21.97
3,2,male,30.74,19.74
4,3,female,21.75,16.12
5,3,male,26.51,12.66


In [53]:
df.groupby(["Pclass", "Sex"], as_index=False)[['Age', 'Fare']].mean()

,Pclass,Sex,Age,Fare
0,1,female,34.61,106.13
1,1,male,41.28,67.23
2,2,female,28.72,21.97
3,2,male,30.74,19.74
4,3,female,21.75,16.12
5,3,male,26.51,12.66


_____________
_____________
**Exercício:** Existe diferença de sobrevivência por portão de embarque? E diferença no preço do ticket? Porque você acha que tem essa diferença?

In [72]:
df.groupby(['Embarked', 'Pclass'])[['Survived','Fare']].agg(['sum', 'mean', 'count'])

Survived                Fare             
                     sum mean count      sum   mean count
Embarked Pclass                                          
C        1            59 0.69    85 8,901.08 104.72    85
         2             9 0.53    17   431.09  25.36    17
         3            25 0.38    66   740.13  11.21    66
Q        1             1 0.50     2   180.00  90.00     2
         2             2 0.67     3    37.05  12.35     3
         3            27 0.38    72   805.20  11.18    72
S        1            74 0.58   127 8,936.34  70.36   127
         2            76 0.46   164 3,333.70  20.33   164
         3            67 0.19   353 5,169.36  14.64   353

In [64]:
# Resposta
df.groupby(["Embarked", 'Pclass'])[["Fare"]].mean()


Fare
Embarked Pclass       
C        1      104.72
         2       25.36
         3       11.21
Q        1       90.00
         2       12.35
         3       11.18
S        1       70.36
         2       20.33
         3       14.64

In [65]:
df.groupby(["Embarked", 'Pclass'])[["Survived", "Age"]].agg(['mean', 'count'])

Survived         Age      
                    mean count  mean count
Embarked Pclass                           
C        1          0.69    85 38.03    74
         2          0.53    17 22.77    15
         3          0.38    66 20.74    41
Q        1          0.50     2 38.50     2
         2          0.67     3 43.50     2
         3          0.38    72 25.94    24
S        1          0.58   127 38.15   108
         2          0.46   164 30.39   156
         3          0.19   353 25.70   290

In [73]:
df.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.00,-0.01,-0.04,0.04,-0.06,-0.00,0.01
Survived,-0.01,1.00,-0.34,-0.08,-0.04,0.08,0.26
Pclass,-0.04,-0.34,1.00,-0.37,0.08,0.02,-0.55
Age,0.04,-0.08,-0.37,1.00,-0.31,-0.19,0.10
SibSp,-0.06,-0.04,0.08,-0.31,1.00,0.41,0.16
Parch,-0.00,0.08,0.02,-0.19,0.41,1.00,0.22
Fare,0.01,0.26,-0.55,0.10,0.16,0.22,1.00


______________
_____________

E se quiséssemos criar uma coluna nova que contenham o valor médio do Fare por Pclass?

### Criando coluna com dado agregado

Queremos que todas as pessoas da primeira classe tenham o valor 84.15 nessa nova coluna, todas da segunda classe tenham o valor 20.66 e da terceira classe 13.67. <br>
Podemos tentar:

In [74]:
df.groupby('Pclass')[["Fare"]].mean()

,Fare
Pclass,
1,84.15
2,20.66
3,13.68


In [75]:
df["Fare_Mean"] = df.groupby('Pclass')["Fare"].mean()

df.head(7)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C,84.15
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S,20.66
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S,13.68
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S,NaN
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.46,NaN,Q,NaN
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.00,0,0,17463,51.86,E46,S,NaN


Xiiii... deu ruim...
<br>
<br>


#### Transformação dos dados
Ao aplicarmos o método `.transform()` temos como retorno um objeto com o mesmo index do df de origem contendo a a transformação realizada para cada uma das linhas. Dessa forma podemos utilizar esse método e apenas criar uma coluna nova no nosso df.
<br>

Ele será muito **útil na criação de novas features** para os modelos.

In [77]:
df.groupby('Pclass')[["Fare"]].mean()

,Fare
Pclass,
1,84.15
2,20.66
3,13.68


In [78]:
df.groupby('Pclass')[["Fare"]].transform('mean')

,Fare
0,13.68
1,84.15
2,13.68
3,84.15
4,13.68
...,...
886,20.66
887,84.15
888,13.68
889,84.15


In [83]:
df["Fare_Mean"] = df.groupby('Pclass')["Fare"].transform('mean')
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S,13.68
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C,84.15
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S,13.68
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S,84.15
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S,13.68
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.46,NaN,Q,13.68
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.00,0,0,17463,51.86,E46,S,84.15
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.00,3,1,349909,21.07,NaN,S,13.68
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.00,0,2,347742,11.13,NaN,S,13.68
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.00,1,0,237736,30.07,NaN,C,20.66


Podemos aplicar tanto as operações mencionadas na agregação quanto uma função `lambda`:

In [84]:
def mult_2(x):
    return x*2

mult_2(4)

8

In [85]:
funcao_lambda = lambda x: 2*x
funcao_lambda(4)

8

In [86]:
funcao_lambda

<function __main__.<lambda>(x)>

In [87]:
df['variacao_max_min'] = df.groupby('Pclass')[["Fare"]].transform(lambda x: x.max() - x.min())
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean,variacao_max_min
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S,13.68,69.55
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C,84.15,512.33
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S,13.68,69.55
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S,84.15,512.33
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S,13.68,69.55
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.46,NaN,Q,13.68,69.55
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.00,0,0,17463,51.86,E46,S,84.15,512.33
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.00,3,1,349909,21.07,NaN,S,13.68,69.55
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.00,0,2,347742,11.13,NaN,S,13.68,69.55
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.00,1,0,237736,30.07,NaN,C,20.66,73.50


Ou até mesmo passar funções construídas:

In [88]:
def funcao_max_menos_min(x):
    return x.max() - x.min()

In [89]:
df.groupby('Pclass')[["Fare"]].transform(funcao_max_menos_min)

,Fare
0,69.55
1,512.33
2,69.55
3,512.33
4,69.55
...,...
886,73.50
887,512.33
888,69.55
889,512.33


Também podemos preencher os valores nulos com a média de cada grupo

In [90]:
# verificando quantidade de nulos por coluna
df.isna().sum()

PassengerId           0
Survived              0
Pclass                0
Name                  0
Sex                   0
Age                 177
SibSp                 0
Parch                 0
Ticket                0
Fare                  0
Cabin               687
Embarked              2
Fare_Mean             0
variacao_max_min      0
dtype: int64

Para preencher os nulos utilizaremos o método `.fillna()` que vimos em aula:

In [91]:
df[['Age']].mean()

Age   29.70
dtype: float64

In [92]:
df.Age.fillna(df.Age.mean())

0     22.00
1     38.00
2     26.00
3     35.00
4     35.00
       ... 
886   27.00
887   19.00
888   29.70
889   26.00
890   32.00
Name: Age, Length: 891, dtype: float64

In [93]:
df['Age_sem_nulo'] = df.groupby(['Pclass'])[['Age']].transform(lambda x: x.fillna(x.mean()))

In [94]:
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean,variacao_max_min,Age_sem_nulo
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S,13.68,69.55,22.00
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C,84.15,512.33,38.00
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S,13.68,69.55,26.00
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S,84.15,512.33,35.00
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S,13.68,69.55,35.00
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.46,NaN,Q,13.68,69.55,25.14
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.00,0,0,17463,51.86,E46,S,84.15,512.33,54.00
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.00,3,1,349909,21.07,NaN,S,13.68,69.55,2.00
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.00,0,2,347742,11.13,NaN,S,13.68,69.55,27.00
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.00,1,0,237736,30.07,NaN,C,20.66,73.50,14.00


In [95]:
df.isna().sum()

PassengerId           0
Survived              0
Pclass                0
Name                  0
Sex                   0
Age                 177
SibSp                 0
Parch                 0
Ticket                0
Fare                  0
Cabin               687
Embarked              2
Fare_Mean             0
variacao_max_min      0
Age_sem_nulo          0
dtype: int64

In [96]:
# Conferindo o preenchimento de nulos
# idade média por Pclass
df.groupby(['Pclass'])[['Age']].mean()

,Age
Pclass,
1,38.23
2,29.88
3,25.14


In [97]:
# selecionando a parte do df que tem idade nula
df[df.Age.isna()].head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean,variacao_max_min,Age_sem_nulo
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.46,NaN,Q,13.68,69.55,25.14
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.00,NaN,S,20.66,73.50,29.88
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.22,NaN,C,13.68,69.55,25.14
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.22,NaN,C,13.68,69.55,25.14
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.88,NaN,Q,13.68,69.55,25.14
29,30,0,3,"Todoroff, Mr. Lalio",male,NaN,0,0,349216,7.90,NaN,S,13.68,69.55,25.14
31,32,1,1,"Spencer, Mrs. William Augustus (Marie Eugenie)",female,NaN,1,0,PC 17569,146.52,B78,C,84.15,512.33,38.23
32,33,1,3,"Glynn, Miss. Mary Agatha",female,NaN,0,0,335677,7.75,NaN,Q,13.68,69.55,25.14
36,37,1,3,"Mamee, Mr. Hanna",male,NaN,0,0,2677,7.23,NaN,C,13.68,69.55,25.14
42,43,0,3,"Kraeff, Mr. Theodor",male,NaN,0,0,349253,7.90,NaN,C,13.68,69.55,25.14


_________________________
_________________________
**Exercício:** Crie uma coluna com a média de Fare e outra com a média de idade para cada classe da coluna Survived. Você consegue fazer isso de uma única vez?

In [126]:
df.groupby(['Survived'])[['Fare', 'Age']].mean()

,Fare,Age
Survived,,
0,22.12,30.63
1,48.40,28.34


In [127]:
# Resposta
df[['media_Fare', 'media_Age']] = df.groupby(['Survived'])[['Fare', 'Age']].transform('mean')

In [130]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean,variacao_max_min,Age_sem_nulo,media_Fare,media_Age
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S,13.68,69.55,22.00,22.12,30.63
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C,84.15,512.33,38.00,48.40,28.34
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S,13.68,69.55,26.00,48.40,28.34
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S,84.15,512.33,35.00,48.40,28.34
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S,13.68,69.55,35.00,22.12,30.63


_________________________
_________________________

## Cruzamento e concatenação de bases

Também é possível fazer **cruzamento de bases** com o pandas. 

Pra quem conhece SQL: esses são os joins!

Pra quem conhece Excel: essa é uma forma de fazer o procv/vlookup!

Vamos supor que temos as notas de duas provas dos alunos separas em sheets diferentes do excel e queremos juntar essa notas em um único df.

In [131]:
# ler os dados de diferentes sheets do mesmo excel "notas.xlsx"
df1 = pd.read_excel("notas.xlsx", sheet_name="notas1")
df2 = pd.read_excel("notas.xlsx", sheet_name="notas2")

In [132]:
df1

,RA,aluno,prova1
0,1,joão,10
1,4,leia,10
2,2,maria,9
3,3,han,8
4,5,luke,7
5,7,obi wan,10


In [133]:
df2

,RA,aluno,prova2
0,1,joão,10
1,4,leia,10
2,2,maria,9
3,3,han,8
4,5,luke,7
5,6,anakin,10


Repare que temos alunos distintos nos dois df

Diferentes tipos de join

<img src="join_exemplo2.png" />
Fonte: https://towardsdatascience.com/python-pandas-dataframe-join-merge-and-concatenate-84985c29ef78

O pandas possui dois métodos específicos para trabalharmos com o join de colunas entre df: `.merge()` e `.join()`. O `.merge()` fornece mais flexibilidade de trabalho e iremos utilizar e ele.

### pd.merge()
pd.merge(
    left,
    right,
    how="inner",
    on=None,
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y")
)

In [145]:
df3 = df1.merge(df2[['aluno', 'prova2']], how = 'left', on = 'aluno')
df3['prova2'].fillna(10, inplace = True)
df3

,RA,aluno,prova1,prova2
0,1,joão,10,10.00
1,4,leia,10,10.00
2,2,maria,9,9.00
3,3,han,8,8.00
4,5,luke,7,7.00
5,7,obi wan,10,10.00


In [134]:
df1.merge(df2[['RA','prova2']], how="outer", on="RA")

,RA,aluno,prova1,prova2
0,1,joão,10.00,10.00
1,4,leia,10.00,10.00
2,2,maria,9.00,9.00
3,3,han,8.00,8.00
4,5,luke,7.00,7.00
5,7,obi wan,10.00,NaN
6,6,NaN,NaN,10.00


In [135]:
df1.merge(df2, how="outer", on="RA", suffixes=['_prova1','_prova2'])

,RA,aluno_prova1,prova1,aluno_prova2,prova2
0,1,joão,10.00,joão,10.00
1,4,leia,10.00,leia,10.00
2,2,maria,9.00,maria,9.00
3,3,han,8.00,han,8.00
4,5,luke,7.00,luke,7.00
5,7,obi wan,10.00,NaN,NaN
6,6,NaN,NaN,anakin,10.00


In [136]:
df1.merge(df2, how="outer", on=["RA", "aluno"])

,RA,aluno,prova1,prova2
0,1,joão,10.00,10.00
1,4,leia,10.00,10.00
2,2,maria,9.00,9.00
3,3,han,8.00,8.00
4,5,luke,7.00,7.00
5,7,obi wan,10.00,NaN
6,6,anakin,NaN,10.00


In [146]:
df1.merge(df2, how="left", on=["RA", "aluno"])

,RA,aluno,prova1,prova2
0,1,joão,10,10.00
1,4,leia,10,10.00
2,2,maria,9,9.00
3,3,han,8,8.00
4,5,luke,7,7.00
5,7,obi wan,10,NaN


In [147]:
df1.merge(df2, how="right", on=["RA", "aluno"])

,RA,aluno,prova1,prova2
0,1,joão,10.00,10
1,4,leia,10.00,10
2,2,maria,9.00,9
3,3,han,8.00,8
4,5,luke,7.00,7
5,6,anakin,NaN,10


In [148]:
df1.merge(df2, how="inner", on=["RA", "aluno"])

,RA,aluno,prova1,prova2
0,1,joão,10,10
1,4,leia,10,10
2,2,maria,9,9
3,3,han,8,8
4,5,luke,7,7


### pd.concat()
Diferente do `.merge()` e `.join()` que operam apenas com colunas, com o `.concat()` podemos especificar se queremos **concatenar em linhas ou colunas**.
Na concatenação de colunas o `.concat()` somente considera o index dos df e, por isso, não podemos especificar colunas como feito com o `.merge()`.

`pd.concat(
    objs,
    axis=0,
    join="outer",
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=True,
)`


In [149]:
pd.concat([df1, df2], axis=1, join="inner")

,RA,aluno,prova1,RA,aluno,prova2
0,1,joão,10,1,joão,10
1,4,leia,10,4,leia,10
2,2,maria,9,2,maria,9
3,3,han,8,3,han,8
4,5,luke,7,5,luke,7
5,7,obi wan,10,6,anakin,10


Repare que ao concatenar diretamente pelo index ele juntou o aluno obi wan com o anakin. 

Ao concatenar dois df nas linhas, o `.concat()` irá considerar o nome das colunas. Se temos colunas com nomes distintos e utilizamos o parâmetro join='inner', ele irá ignorar essas colunas: 

In [150]:
pd.concat([df1, df2], axis=0, join="inner")

,RA,aluno
0,1,joão
1,4,leia
2,2,maria
3,3,han
4,5,luke
5,7,obi wan
0,1,joão
1,4,leia
2,2,maria
3,3,han


Para que ele considere todas as colunas utilizamos o argumento 
```python 
join="outer" 
```

In [151]:
pd.concat([df1, df2], join="outer")

,RA,aluno,prova1,prova2
0,1,joão,10.00,NaN
1,4,leia,10.00,NaN
2,2,maria,9.00,NaN
3,3,han,8.00,NaN
4,5,luke,7.00,NaN
5,7,obi wan,10.00,NaN
0,1,joão,NaN,10.00
1,4,leia,NaN,10.00
2,2,maria,NaN,9.00
3,3,han,NaN,8.00


## Exercícios

1. Considere a existência de três tabelas distintas:
* customer.csv : Possui a informação dos clientes em duas colunas: customer id  customer name
* products.csv : Conté informação dos produtos vendidos pela empresa em três colunas - p_id (product id), product (name) e price
* sales.csv : Contém informações das vendas realizadas em seis colunas - sale_id, c_id (customer id), p_id (product_id), qty (quantity sold), store (name)

Conhecendo as bases e utilizando os métodos de cruzamento e concatenação de bases responda:


a) Quais produtos não foram vendidos?

In [152]:
import pandas as pd 
sales = pd.read_csv("data/sales.csv")
products = pd.read_csv("data/products.csv")
customer = pd.read_csv("data/customer.csv")

In [166]:
products

,p_id,product,price
0,1,Hard Disk,80
1,2,RAM,90
2,3,Monitor,75
3,4,CPU,55
4,5,Keyboard,20
5,6,Mouse,10
6,7,Motherboard,50
7,8,Power supply,20


In [156]:
sales

,sale_id,c_id,p_id,product,qty,store
0,1,2,3,Monitor,2,ABC
1,2,2,4,CPU,1,DEF
2,3,1,3,Monitor,3,ABC
3,4,4,2,RAM,2,DEF
4,5,2,3,Monitor,3,ABC
5,6,3,3,Monitor,2,DEF
6,7,2,2,RAM,3,ABC
7,8,3,2,RAM,2,DEF
8,9,2,3,Monitor,2,ABC


In [185]:
products.merge(sales[['product','qty']], how = 'left', on = 'product').groupby('product')[['qty']].mean()

,qty
product,
CPU,1.00
Hard Disk,NaN
Keyboard,NaN
Monitor,2.40
Motherboard,NaN
Mouse,NaN
Power supply,NaN
RAM,2.33


In [323]:
products.merge(sales[['product','qty']], how = 'left', on = 'product').groupby('product')[['qty']].apply(lambda x: x.mean().isna())[['qty']]

,qty
product,
CPU,False
Hard Disk,True
Keyboard,True
Monitor,False
Motherboard,True
Mouse,True
Power supply,True
RAM,False


In [209]:
df2 = products.merge(sales[['product','qty']], how = 'left', on = 'product')
df2

,p_id,product,price,qty
0,1,Hard Disk,80,NaN
1,2,RAM,90,2.00
2,2,RAM,90,3.00
3,2,RAM,90,2.00
4,3,Monitor,75,2.00
5,3,Monitor,75,3.00
6,3,Monitor,75,3.00
7,3,Monitor,75,2.00
8,3,Monitor,75,2.00
9,4,CPU,55,1.00


In [229]:
df2.loc[df2['qty'].isna(),:][['product']]

,product
0,Hard Disk
10,Keyboard
11,Mouse
12,Motherboard
13,Power supply


In [224]:
list(df2.loc[df2['qty'].isna(),:]['product'].values)

['Hard Disk', 'Keyboard', 'Mouse', 'Motherboard', 'Power supply']

In [207]:
df2 = products.merge(sales[['product','qty']], how = 'left', on = 'product').groupby('product')[['qty']].mean()
list(df2.loc[df2['qty'].isna(),:].index)

['Hard Disk', 'Keyboard', 'Motherboard', 'Mouse', 'Power supply']

In [230]:
import pandas as pd 
sales = pd.read_csv("data/sales.csv") 
products = pd.read_csv("data/products.csv") 

# Faz um merge priorizando as informações da tabela de produtos
my_data = pd.merge(sales, products, on=['p_id','product'],how='right')
 
# Seleciona todos as linhas em que o sale_id é nan
my_data = my_data[my_data['sale_id'].isna()]

# Seleciona os produtos que não foram vendidos
print(my_data.loc[:,'product'])

0        Hard Disk
10        Keyboard
11           Mouse
12     Motherboard
13    Power supply
Name: product, dtype: object


b) Quantos clientes não realizaram uma compra? 

In [232]:
customer

,c_id,Customer
0,1,Rabi
1,2,Raju
2,3,Alex
3,4,Rani
4,5,King
5,7,Ronn
6,8,Jem
7,9,Tom


In [233]:
sales

,sale_id,c_id,p_id,product,qty,store
0,1,2,3,Monitor,2,ABC
1,2,2,4,CPU,1,DEF
2,3,1,3,Monitor,3,ABC
3,4,4,2,RAM,2,DEF
4,5,2,3,Monitor,3,ABC
5,6,3,3,Monitor,2,DEF
6,7,2,2,RAM,3,ABC
7,8,3,2,RAM,2,DEF
8,9,2,3,Monitor,2,ABC


In [246]:
customer.merge(sales, on = 'c_id', how = 'left')['sale_id'].isna().sum()

4

In [247]:
customers = pd.read_csv("data/customer.csv") 

# Faz um merge priorizando as informações da tabela de customers
my_data = pd.merge(sales, customers, on='c_id', how='right')

# Seleciona todos as linhas em que o sale_id é nan
my_data = my_data[my_data['sale_id'].isna()] 

# Seleciona os clientes que não realizaram compras
print(my_data.loc[:,'Customer']) 

9     King
10    Ronn
11     Jem
12     Tom
Name: Customer, dtype: object


c) Liste a quantidade vendida e o faturamento de cada produto 

In [250]:
products

,p_id,product,price
0,1,Hard Disk,80
1,2,RAM,90
2,3,Monitor,75
3,4,CPU,55
4,5,Keyboard,20
5,6,Mouse,10
6,7,Motherboard,50
7,8,Power supply,20


In [265]:
df = sales.merge(products[['p_id', 'price']], how = 'left', on = 'p_id')
df['faturamento'] = df['qty']*df['price']
df.groupby('product')[['qty', 'faturamento']].sum()

,qty,faturamento
product,,
CPU,1,55
Monitor,12,900
RAM,7,630


In [288]:
df2 = sales.merge(products[['p_id', 'price']], how = 'left', on = 'p_id')
df2['faturamento'] = df2['qty']*df2['price']
df2.groupby('product')[['qty', 'faturamento']].sum()

,qty,faturamento
product,,
CPU,1,55
Monitor,12,900
RAM,7,630


In [324]:
df2 = sales.merge(products[['p_id', 'price']], how = 'left', on = 'p_id')
df2['faturamento'] = df2['qty']*df2['price']
df2.groupby('product')[['qty', 'faturamento']].apply(lambda x: x.sum())

,qty,faturamento
product,,
CPU,1,55
Monitor,12,900
RAM,7,630


In [266]:
my_sum=pd.merge(sales, products, how='left', on=['p_id','product'])

# Calcula valor total de cada sale_id
my_sum['total_sale'] = my_sum['qty']*my_sum['price']

# Agrupa por produto e soma a quantidade vendida e o valor total da compra
my_sum.groupby(['product'])[['qty', 'total_sale']].sum()

,qty,total_sale
product,,
CPU,1,55
Monitor,12,900
RAM,7,630


d) Liste a quantidade vendida de cada produto por loja

In [347]:
sales.merge(products[['p_id', 'price']], how = 'left', on = 'p_id').groupby(['product', 'store'])[['qty']].sum().reset_index()

,product,store,qty
0,CPU,DEF,1
1,Monitor,ABC,10
2,Monitor,DEF,2
3,RAM,ABC,3
4,RAM,DEF,4


In [333]:
my_sale = sales.groupby(['product','p_id', 'store'])[['qty']].sum().reset_index()
my_sale

,product,p_id,store,qty
0,CPU,4,DEF,1
1,Monitor,3,ABC,10
2,Monitor,3,DEF,2
3,RAM,2,ABC,3
4,RAM,2,DEF,4


e) Qual loja teve maior faturamento?

In [356]:
df2 = sales.merge(products[['p_id', 'price']], how = 'left', on = 'p_id')
df2['faturamento'] = df2['qty']*df2['price']
df2 = df2.groupby(['store'])[['qty', 'faturamento']].sum().reset_index()
df2.sort_values(by = ['faturamento'], ascending = False).head(1)

,store,qty,faturamento
0,ABC,13,1020


In [344]:
my_sum.groupby(['store'])[['total_sale']].sum().sort_values('total_sale').tail(1)

,total_sale
store,
ABC,1020


f) Qual produto foi o mais vendido?

In [357]:
df2 = sales.merge(products[['p_id', 'price']], how = 'left', on = 'p_id')
df2['faturamento'] = df2['qty']*df2['price']
df2 = df2.groupby(['product'])[['qty', 'faturamento']].sum().reset_index()
df2.sort_values(by = ['faturamento'], ascending = False).head(1)

,product,qty,faturamento
1,Monitor,12,900


In [343]:
my_sum.groupby(['product'])[['qty']].sum().sort_values('qty').tail(1)

,qty
product,
Monitor,12


## Referências
https://pandas.pydata.org/docs/user_guide/groupby.html <br>
https://pandas.pydata.org/docs/user_guide/merging.html <br> 
https://towardsdatascience.com/python-pandas-dataframe-join-merge-and-concatenate-84985c29ef78 <br>
[When to use pandas transform function](https://towardsdatascience.com/when-to-use-pandas-transform-function-df8861aa0dcf) <br>
[Compara a performance entre várias formas de iterar em um df. Vai desde o for até apply e transform](https://youtu.be/rsyvErL0Bo8) <br>

## Material extra

### Outros parâmetros do groupby por default
* as_index
* sort
* dropna # exclui nans nas keys

<br> Em todas o default do python é True <br>
df.groupby('Pclass', sort=False)["Fare"].mean()

In [ ]:
# dropna
df_list = [[1, 2, 3], [1, None, 4], [2, 1, 3], [1, 2, 2]]
df_dropna = pd.DataFrame(df_list, columns=["a", "b", "c"])
df_dropna

In [ ]:
# Default ``dropna`` is set to True, which will exclude NaNs in keys
df_dropna.groupby(by=["b"], dropna=True).sum()

In [ ]:
df_dropna.groupby(by=["b"], dropna=False).sum()

Repare que podemos chamar qualquer função do `pd.Series` ou  do `numpy`

In [ ]:
df.groupby(["Survived"]).mean()

### Função Lambda
Uma função lambda nada mais é que uma **forma alternativa de declarar uma função**, de um jeito mais direto

In [ ]:
# função que retorna o dobro de um número usando def
def dobro(x):
    
    return 2*x

dobro(2)

In [ ]:
# função que retorna o dobro de um número usando lambda x
faz_dobro = lambda x: 2*x

In [ ]:
faz_dobro(6)

### Apply
O método `.apply()` recebe uma função como input e aplica ela para todo o df como se fosse um loop. Se você quiser que essa função seja aplicada ao longo das colunas deve considerar axis=0 e ao longo das linhas axis=1)

In [358]:
df.groupby(['Pclass']).apply(lambda x: x.describe())

PassengerId  Survived  Pclass    Age  SibSp  Parch   Fare  \
Pclass                                                                    
1      count       216.00    216.00  216.00 186.00 216.00 216.00 216.00   
       mean        461.60      0.63    1.00  38.23   0.42   0.36  84.15   
       std         246.74      0.48    0.00  14.80   0.61   0.69  78.38   
       min           2.00      0.00    1.00   0.92   0.00   0.00   0.00   
       25%         270.75      0.00    1.00  27.00   0.00   0.00  30.92   
       50%         472.00      1.00    1.00  37.00   0.00   0.00  60.29   
       75%         670.50      1.00    1.00  49.00   1.00   0.00  93.50   
       max         890.00      1.00    1.00  80.00   3.00   4.00 512.33   
2      count       184.00    184.00  184.00 173.00 184.00 184.00 184.00   
       mean        445.96      0.47    2.00  29.88   0.40   0.38  20.66   
       std         250.85      0.50    0.00  14.00   0.60   0.69  13.42   
       min          10.00      0.00    2.00   0.67   0.00   0.00   0.00   
       25%         234.50      0.00    2.00  23.00   0.00   0.00  13.00   
       50%         435.50      0.00    2.00  29.00   0.00   0.00  14.25   
       75%         668.00      1.00    2.00  36.00   1.00   1.00  26.00   
       max         887.00      1.00    2.00  70.00   3.00   3.00  73.50   
3      count       491.00    491.00  491.00 355.00 491.00 491.00 491.00   
       mean        439.15      0.24    3.00  25.14   0.62   0.39  13.68   
       std         264.44      0.43    0.00  12.50   1.37   0.89  11.78   
       min           1.00      0.00    3.00   0.42   0.00   0.00   0.00   
       25%         200.00      0.00    3.00  18.00   0.00   0.00   7.75   
       50%         432.00      0.00    3.00  24.00   0.00   0.00   8.05   
       75%         666.50      0.00    3.00  32.00   1.00   0.00  15.50   
       max         891.00      1.00    3.00  74.00   8.00   6.00  69.55   

              Fare_original  Fare_variacao  
Pclass                                      
1      count         216.00         216.00  
       mean           84.15           0.00  
       std            78.38          78.38  
       min             0.00         -84.15  
       25%            30.92         -53.23  
       50%            60.29         -23.87  
       75%            93.50           9.35  
       max           512.33         428.17  
2      count         184.00         184.00  
       mean           20.66           0.00  
       std            13.42          13.42  
       min             0.00         -20.66  
       25%            13.00          -7.66  
       50%            14.25          -6.41  
       75%            26.00           5.34  
       max            73.50          52.84  
3      count         491.00         491.00  
       mean           13.68           0.00  
       std            11.78          11.78  
       min             0.00         -13.68  
       25%             7.75          -5.93  
       50%             8.05          -5.63  
       75%            15.50           1.82  
       max            69.55          55.87

Uma grande funcionalidade do pandas é que com o método `apply()` podemos aplicar uma **função** (muitas vezes, uma **função lambda**) a uma coluna ou linha de um DataFrame



Vamos selecionar a coluna de idades...

In [ ]:
df["Age"]

Aplicando uma função lambda **a todos os elementos da coluna**, ou seja, **à todas as linhas da tabela, daquela coluna específica**:

Tomando cada idade + 2, usando a função lambda definida.

Essa função lambda é equivalente a:

```python

def funcao(x):

    return x + 2
```

In [273]:
df["Age"].apply(lambda x: x + 2)

0     24.00
1     40.00
2     28.00
3     37.00
4     37.00
       ... 
886   29.00
887   21.00
888     NaN
889   28.00
890   34.00
Name: Age, Length: 891, dtype: float64

In [274]:
def funcao(x):
    return x + 2

df.Age.apply(funcao)

0     24.00
1     40.00
2     28.00
3     37.00
4     37.00
       ... 
886   29.00
887   21.00
888     NaN
889   28.00
890   34.00
Name: Age, Length: 891, dtype: float64

In [275]:
df.Age.transform(funcao)

0     24.00
1     40.00
2     28.00
3     37.00
4     37.00
       ... 
886   29.00
887   21.00
888     NaN
889   28.00
890   34.00
Name: Age, Length: 891, dtype: float64

Um outro exemplo:

In [276]:
# função: transforma todos os números em string, e concatena "!!!!!!!!!" à string
df["Age"].apply(lambda x: str(x) + "!!!!!!!!!")

0      22.0!!!!!!!!!
1      38.0!!!!!!!!!
2      26.0!!!!!!!!!
3      35.0!!!!!!!!!
4      35.0!!!!!!!!!
           ...      
886    27.0!!!!!!!!!
887    19.0!!!!!!!!!
888     nan!!!!!!!!!
889    26.0!!!!!!!!!
890    32.0!!!!!!!!!
Name: Age, Length: 891, dtype: object

Vamos usar uma função lambda para **extrair o sobrenome** dos nomes dos passageiros

Pra extrarir o sobrenome, note que este está separada do resto do nome por vírgula.

Para perceber isso, dê uma olhada na coluna de nomes:

In [277]:
df["Name"]

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
886                                Montvila, Rev. Juozas
887                         Graham, Miss. Margaret Edith
888             Johnston, Miss. Catherine Helen "Carrie"
889                                Behr, Mr. Karl Howell
890                                  Dooley, Mr. Patrick
Name: Name, Length: 891, dtype: object

Portanto, podemos usar a função para strings `split(",")`, com quebra na vírgula, e depois selecionar o primeiro elemento da lista gerada!

Vamos aproveitar e **criar uma nova coluna da base**, com os sobrenomes!

In [278]:
df["Surname"] = df["Name"].apply(lambda x: x.split(",")[0])

In [279]:
df["Surname"]

0         Braund
1        Cumings
2      Heikkinen
3       Futrelle
4          Allen
         ...    
886     Montvila
887       Graham
888     Johnston
889         Behr
890       Dooley
Name: Surname, Length: 891, dtype: object

### Apply com funções

E se quisessemos comparar o quanto cada passageiro pagou a mais ou a menos da média do Fare?

In [280]:
def f(group):
    return pd.DataFrame({'Fare_original': group,
                         'Fare_variacao': group - group.mean()})

df[['Fare_original','Fare_variacao']] = df.groupby('Pclass')['Fare'].apply(f)

In [281]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Fare_original,Fare_variacao
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S,Braund,7.25,-6.43
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C,Cumings,71.28,-12.87
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S,Heikkinen,7.92,-5.75
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S,Futrelle,53.10,-31.05
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S,Allen,8.05,-5.63


#### Transform X Apply
Com uma função de agregação o `.transform()` retorna um df que tem a mesma quantidade de linhas que o df original enquanto o `.apply` retorna o agregado por grupos.

### Filtros
O filtro retorna apenas um subset do nosso df. Aqui podemos aplicar filtros mais elaborados do que os vistos na última aula. <br>
Podemos, por exemplo, eliminar categorias do df que possuem apenas alguns elementos:

In [ ]:
df.SibSp.value_counts()

In [ ]:
df.shape

In [ ]:
def filter_func(x):
    return x['Fare'] - x.Fare_Mean < 100

# df_filter = df.groupby(['SibSp']).filter(lambda x: filter_func(x))

df_filter = df.groupby(['SibSp']).filter(lambda x: len(x) >20)
df_filter.shape

In [ ]:
df_filter.SibSp.value_counts()

Vamos supor que antes de afundar o titanic, o time de hapiness quisesse promover uma jogatina para os grupos (segmentado por classe e sexo) que possuem idade média acima de 30 anos.

In [ ]:
df.groupby(['Pclass','Sex'])[['Age']].mean()

como podemos filtrar nosso df para termos apenas os passageiros que pertecem a essas segmentações escolhidas?

In [ ]:
df.groupby(['Pclass','Sex']).filter(lambda x: x['Age'].mean()>30)